# Linear Model
This model is the same as the model found in examples/predictors/linear. It has been copied and moved into examples/predictors/ryan_predictor so changes can be made while leaving the original in tact. This notebook also runs top to bottom with no errors (unlike the one with the example linear predictor)
### Improvements
For the next iteration try using some thing like the 'New Cases' column but make it a "new cases as a percentage of current cases" column instead like:
`case_percent_increase = 1 + (case_count_today - case_count_yesterday) / case_count_yesterday`

In [1]:
import pickle
import os
import urllib.request
from sklearn.linear_model import Lasso
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

## Validation Data?

In [2]:
path_to_ips_file="validation/data/2020-09-30_historical_ip.csv"
input_file = pd.read_csv(path_to_ips_file, low_memory=False)

In [3]:
# input_file[input_file['CountryName'] == 'United States']
input_file[input_file['RegionName'] == 'California']

,CountryName,RegionName,Date,C1_School closing,C2_Workplace closing,C3_Cancel public events,C4_Restrictions on gatherings,C5_Close public transport,C6_Stay at home requirements,C7_Restrictions on internal movement,C8_International travel controls,H1_Public information campaigns,H2_Testing policy,H3_Contact tracing,H6_Facial Coverings
50690,United States,California,2020-01-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
50691,United States,California,2020-01-02,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
50692,United States,California,2020-01-03,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
50693,United States,California,2020-01-04,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
50694,United States,California,2020-01-05,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50959,United States,California,2020-09-26,3.0,2.0,2.0,4.0,0.0,1.0,1.0,3.0,2.0,3.0,2.0,3.0
50960,United States,California,2020-09-27,3.0,2.0,2.0,4.0,0.0,1.0,1.0,3.0,2.0,3.0,2.0,3.0
50961,United States,California,2020-09-28,3.0,2.0,2.0,4.0,0.0,1.0,1.0,3.0,2.0,3.0,2.0,3.0
50962,United States,California,2020-09-29,3.0,2.0,2.0,4.0,0.0,1.0,1.0,3.0,2.0,3.0,2.0,3.0


## Importing the Training Data

In [4]:
# Main source for the training data
DATA_URL = 'https://raw.githubusercontent.com/OxCGRT/covid-policy-tracker/master/data/OxCGRT_latest.csv'
# Local files
data_path = 'examples/predictors/ryan_predictor/data'
DATA_FILE = data_path + '/OxCGRT_latest.csv'


if not os.path.exists(data_path):
    os.mkdir(data_path)
urllib.request.urlretrieve(DATA_URL, DATA_FILE)

('examples/predictors/ryan_predictor/data/OxCGRT_latest.csv',
 <http.client.HTTPMessage at 0x7fcd3ca78460>)

In [5]:
df = pd.read_csv(DATA_FILE, 
                 parse_dates=['Date'],
                 encoding="ISO-8859-1",
                 dtype={"RegionName": str,
                        "RegionCode": str},
                 error_bad_lines=False)
# df[cases_df['RegionName'] == 'California']

In [6]:
df.columns

Index(['CountryName', 'CountryCode', 'RegionName', 'RegionCode',
       'Jurisdiction', 'Date', 'C1_School closing', 'C1_Flag',
       'C2_Workplace closing', 'C2_Flag', 'C3_Cancel public events', 'C3_Flag',
       'C4_Restrictions on gatherings', 'C4_Flag', 'C5_Close public transport',
       'C5_Flag', 'C6_Stay at home requirements', 'C6_Flag',
       'C7_Restrictions on internal movement', 'C7_Flag',
       'C8_International travel controls', 'E1_Income support', 'E1_Flag',
       'E2_Debt/contract relief', 'E3_Fiscal measures',
       'E4_International support', 'H1_Public information campaigns',
       'H1_Flag', 'H2_Testing policy', 'H3_Contact tracing',
       'H4_Emergency investment in healthcare', 'H5_Investment in vaccines',
       'H6_Facial Coverings', 'H6_Flag', 'H7_Vaccination policy', 'H7_Flag',
       'M1_Wildcard', 'ConfirmedCases', 'ConfirmedDeaths', 'StringencyIndex',
       'StringencyIndexForDisplay', 'StringencyLegacyIndex',
       'StringencyLegacyIndexForDispla

In [7]:
HYPOTHETICAL_SUBMISSION_DATE = np.datetime64("2020-07-31")
df = df[df.Date <= HYPOTHETICAL_SUBMISSION_DATE]

In [8]:
# Add RegionID column that combines CountryName and RegionName for easier manipulation of data
df['GeoID'] = df['CountryName'] + '__' + df['RegionName'].astype(str)

In [9]:
# Add new cases column
df['NewCases'] = df.groupby('GeoID').ConfirmedCases.diff().fillna(0)

In [10]:
# Keep only columns of interest
id_cols = ['CountryName',
           'RegionName',
           'GeoID',
           'Date']
cases_col = ['NewCases']
npi_cols = ['C1_School closing',
            'C2_Workplace closing',
            'C3_Cancel public events',
            'C4_Restrictions on gatherings',
            'C5_Close public transport',
            'C6_Stay at home requirements',
            'C7_Restrictions on internal movement',
            'C8_International travel controls',
            'H1_Public information campaigns',
            'H2_Testing policy',
            'H3_Contact tracing',
            'H6_Facial Coverings']
df = df[id_cols + cases_col + npi_cols]

In [11]:
# Fill any missing case values by interpolation and setting NaNs to 0
df.update(df.groupby('GeoID').NewCases.apply(
    lambda group: group.interpolate()).fillna(0))

In [12]:
# Fill any missing NPIs by assuming they are the same as previous day
for npi_col in npi_cols:
    df.update(df.groupby('GeoID')[npi_col].ffill().fillna(0))

In [13]:
df

,CountryName,RegionName,GeoID,Date,NewCases,C1_School closing,C2_Workplace closing,C3_Cancel public events,C4_Restrictions on gatherings,C5_Close public transport,C6_Stay at home requirements,C7_Restrictions on internal movement,C8_International travel controls,H1_Public information campaigns,H2_Testing policy,H3_Contact tracing,H6_Facial Coverings
0,Aruba,NaN,Aruba__nan,2020-01-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Aruba,NaN,Aruba__nan,2020-01-02,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Aruba,NaN,Aruba__nan,2020-01-03,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Aruba,NaN,Aruba__nan,2020-01-04,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Aruba,NaN,Aruba__nan,2020-01-05,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92693,Zimbabwe,NaN,Zimbabwe__nan,2020-07-27,192.0,3.0,1.0,2.0,3.0,1.0,2.0,2.0,4.0,2.0,1.0,1.0,4.0
92694,Zimbabwe,NaN,Zimbabwe__nan,2020-07-28,113.0,3.0,1.0,2.0,3.0,1.0,2.0,2.0,4.0,2.0,1.0,1.0,4.0
92695,Zimbabwe,NaN,Zimbabwe__nan,2020-07-29,62.0,3.0,1.0,2.0,3.0,1.0,2.0,2.0,4.0,2.0,1.0,1.0,4.0
92696,Zimbabwe,NaN,Zimbabwe__nan,2020-07-30,213.0,3.0,1.0,2.0,3.0,1.0,2.0,2.0,4.0,2.0,1.0,1.0,4.0


## Making the Model

In [14]:
# Set number of past days to use to make predictions
nb_lookback_days = 30

# Create training data across all countries for predicting one day ahead
X_cols = cases_col + npi_cols
y_col = cases_col
X_samples = []
y_samples = []
geo_ids = df.GeoID.unique()
for g in geo_ids:
    gdf = df[df.GeoID == g]
    all_case_data = np.array(gdf[cases_col])
    all_npi_data = np.array(gdf[npi_cols])

    # Create one sample for each day where we have enough data
    # Each sample consists of cases and npis for previous nb_lookback_days
    nb_total_days = len(gdf)
    for d in range(nb_lookback_days, nb_total_days - 1):
        X_cases = all_case_data[d-nb_lookback_days:d]

        # Take negative of npis to support positive
        # weight constraint in Lasso.
        X_npis = -all_npi_data[d - nb_lookback_days:d]

        # Flatten all input data so it fits Lasso input format.
        X_sample = np.concatenate([X_cases.flatten(),
                                   X_npis.flatten()])
        y_sample = all_case_data[d + 1]
        X_samples.append(X_sample)
        y_samples.append(y_sample)

X_samples = np.array(X_samples)
y_samples = np.array(y_samples).flatten()

In [15]:
# Helpful function to compute mae
def mae(pred, true):
    return np.mean(np.abs(pred - true))

In [16]:
# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X_samples,
                                                    y_samples,
                                                    test_size=0.2,
                                                    random_state=301)

In [17]:
# Create and train Lasso model.
# Set positive=True to enforce assumption that cases are positively correlated
# with future cases and npis are negatively correlated.
model = Lasso(alpha=0.1,
              precompute=True,
              max_iter=10000,
              positive=True,
              selection='random')
# Fit model
model.fit(X_train, y_train)

Lasso(alpha=0.1, max_iter=10000, positive=True, precompute=True,
      selection='random')

In [18]:
# Evaluate model
train_preds = model.predict(X_train)
train_preds = np.maximum(train_preds, 0) # Don't predict negative cases
print('Train MAE:', mae(train_preds, y_train))

test_preds = model.predict(X_test)
test_preds = np.maximum(test_preds, 0) # Don't predict negative cases
print('Test MAE:', mae(test_preds, y_test))

Train MAE: 144.7601788221988
Test MAE: 144.08349848629373


In [19]:
# Inspect the learned feature coefficients for the model
# to see what features it's paying attention to.

# Give names to the features
x_col_names = []
for d in range(-nb_lookback_days, 0):
    x_col_names.append('Day ' + str(d) + ' ' + cases_col[0])
for d in range(-nb_lookback_days, 1):
    for col_name in npi_cols:
        x_col_names.append('Day ' + str(d) + ' ' + col_name)

# View non-zero coefficients
for (col, coeff) in zip(x_col_names, list(model.coef_)):
    if coeff != 0.:
        print(col, coeff)
print('Intercept', model.intercept_)

Day -7 NewCases 0.04427498410667145
Day -6 NewCases 0.38681740630986006
Day -5 NewCases 0.255236242080438
Day -4 NewCases 0.060793614445187084
Day -3 NewCases 0.034418467377617605
Day -2 NewCases 0.09717820615503987
Day -1 NewCases 0.19204393599799705
Day -26 C6_Stay at home requirements 7.407936704213173
Day -22 C2_Workplace closing 4.830702640183979
Day -21 C2_Workplace closing 8.084965487066011
Intercept 29.19172101781254


In [20]:
# Save model to file

model_path = 'examples/predictors/ryan_predictor/model'

if not os.path.exists(model_path):
    os.mkdir(model_path)
with open(model_path + '/model.pkl', 'wb') as model_file:
    pickle.dump(model, model_file)

## Evaluating the Model

In [21]:
# Reload the module to get the latest changes
from examples.predictors.linear import predict
from importlib import reload
reload(predict)
from examples.predictors.linear.predict import predict_df

In [22]:
%%time
path_to_ips_file="validation/data/2020-09-30_historical_ip.csv"
preds_df = predict_df("2020-08-01", "2020-08-31", path_to_ips_file, verbose=True)


Predicting for Aruba__nan
2020-08-01: 68.35968479048337
2020-08-02: 82.45764545532413
2020-08-03: 91.43898017975067
2020-08-04: 92.05220592954011
2020-08-05: 93.00589946177968
2020-08-06: 109.57223531002779
2020-08-07: 143.2191449659547
2020-08-08: 162.14281164026357
2020-08-09: 173.9326055305263
2020-08-10: 181.080316933315
2020-08-11: 190.9202972814637
2020-08-12: 210.0956859028169
2020-08-13: 234.2724301966136
2020-08-14: 253.3739025941018
2020-08-15: 267.874522327582
2020-08-16: 280.31274826625986
2020-08-17: 295.25439425939373
2020-08-18: 315.01479946423706
2020-08-19: 336.64668384654857
2020-08-20: 356.1531049635862
2020-08-21: 373.2204849620747
2020-08-22: 389.6071455104002
2020-08-23: 407.7726941838315
2020-08-24: 428.45251857423153
2020-08-25: 450.01175786223735
2020-08-26: 470.6430951272468
2020-08-27: 482.7514034937121
2020-08-28: 509.0535630727247
2020-08-29: 535.0059431718348
2020-08-30: 558.5235810301602
2020-08-31: 581.7241092023668

Predicting for Afghanistan__nan
2020

In [23]:
# Check the predictions
preds_df.head()

,CountryName,RegionName,Date,PredictedDailyNewCases
213,Aruba,NaN,2020-08-01,68.359685
214,Aruba,NaN,2020-08-02,82.457645
215,Aruba,NaN,2020-08-03,91.438980
216,Aruba,NaN,2020-08-04,92.052206
217,Aruba,NaN,2020-08-05,93.005899


## Validation

This is how the predictor is going to be called during the competition.
!!! PLEASE DO NOT CHANGE THE API !!!

In [24]:
!python examples/predictors/linear/ryan_predict.py -s 2020-08-01 -e 2020-08-04 -ip validation/data/2020-09-30_historical_ip.csv -o examples/predictors/ryan_predictor/predictions/2020-08-01_2020-08-04.csv

Generating predictions from 2020-08-01 to 2020-08-04...
Saved predictions to examples/predictors/ryan_predictor/predictions/2020-08-01_2020-08-04.csv
Done!


In [25]:
!head predictions/2020-08-01_2020-08-04.csv

head: predictions/2020-08-01_2020-08-04.csv: No such file or directory


## Test Cases
We can generate a prediction file. Let's validate a few cases...

In [26]:
import sys
from validation.predictor_validation import validate_submission

def validate(start_date, end_date, ip_file, output_file):
    # First, delete any potential old file
    try:
        os.remove(output_file)
    except OSError:
        pass
    
    # Then generate the prediction, calling the official API
    !python examples/predictors/linear/predict.py -s {start_date} -e {end_date} -ip {ip_file} -o {output_file}
    
    # And validate it
    errors = validate_submission(start_date, end_date, ip_file, output_file)
    if errors:
        for error in errors:
            print(error)
    else:
        print("All good!")

### 4 days, no gap
- All countries and regions
- Official number of cases is known up to start_date
- Intervention Plans are the official ones

In [27]:
validate(start_date="2020-08-01",
         end_date="2020-08-04",
         ip_file="validation/data/2020-09-30_historical_ip.csv",
         output_file="examples/predictors/linear/predictions/val_4_days.csv")

Generating predictions from 2020-08-01 to 2020-08-04...
Saved predictions to examples/predictors/linear/predictions/val_4_days.csv
Done!
All good!


### 1 month in the future
- 2 countries only
- there's a gap between date of last known number of cases and start_date
- For future dates, Intervention Plans contains scenarios for which predictions are requested to answer the question: what will happen if we apply these plans?

In [28]:
%%time
validate(start_date="2021-01-01",
         end_date="2021-01-31",
         ip_file="validation/data/future_ip.csv",
         output_file="examples/predictors/linear/predictions/val_1_month_future.csv")

Generating predictions from 2021-01-01 to 2021-01-31...
Saved predictions to examples/predictors/linear/predictions/val_1_month_future.csv
Done!
All good!
CPU times: user 198 ms, sys: 64.9 ms, total: 263 ms
Wall time: 8.16 s


### 180 days, from a future date, all countries and regions
- Prediction start date is 1 week from now. (i.e. assuming submission date is 1 week from now)  
- Prediction end date is 6 months after start date.  
- Prediction is requested for all available countries and regions.  
- Intervention plan scenario: freeze last known intervention plans for each country and region.  

As the number of cases is not known yet between today and start date, but the model relies on them, the model has to predict them in order to use them.  
This test is the most demanding test. It should take less than 1 hour to generate the prediction file.

In [29]:
from datetime import datetime, timedelta

start_date = datetime.now() + timedelta(days=7)
start_date_str = start_date.strftime('%Y-%m-%d')
end_date = start_date + timedelta(days=180)
end_date_str = end_date.strftime('%Y-%m-%d')
print(f"Start date: {start_date_str}")
print(f"End date: {end_date_str}")

Start date: 2020-12-21
End date: 2021-06-19


In [30]:
from validation.scenario_generator import get_raw_data, generate_scenario, NPI_COLUMNS
DATA_FILE = 'examples/predictors/linear/data/OxCGRT_latest.csv'
latest_df = get_raw_data(DATA_FILE, latest=True)
scenario_df = generate_scenario(start_date_str, end_date_str, latest_df, countries=None, scenario="Freeze")
scenario_file = "examples/predictors/linear/predictions/180_days_future_scenario.csv"
scenario_df.to_csv(scenario_file, index=False)
print(f"Saved scenario to {scenario_file}")

Saved scenario to examples/predictors/linear/predictions/180_days_future_scenario.csv


### Check it

In [31]:
%%time
validate(start_date=start_date_str,
         end_date=end_date_str,
         ip_file=scenario_file,
         output_file="examples/predictors/linear/predictions/val_6_month_future.csv")

Generating predictions from 2020-12-21 to 2021-06-19...
Saved predictions to examples/predictors/linear/predictions/val_6_month_future.csv
Done!
All good!
CPU times: user 8.89 s, sys: 2.15 s, total: 11 s
Wall time: 5min 8s
